In [1]:
# Importando Bibliotecas
import os
from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Carrega variáveis do .env
load_dotenv()

credencial = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
project_id = os.getenv("PROJECT_ID")
table_id = os.getenv("TABLE_FACT")
table_stg = os.getenv("TABLE_STG_OTIMZADO")
table_pedido = os.getenv("TABLE_PEDIDO")
table_pizza = os.getenv("TABLE_PIZZA")
table_date = os.getenv("TABLE_DATE")


In [3]:
# Cria cliente BigQuery
client = bigquery.Client.from_service_account_json(credencial, project=project_id)


query = f"""
SELECT *
FROM `{table_stg}`
"""

In [4]:
# Executa e converte pra DataFrame
resultado = client.query(query)
df = resultado.to_dataframe()

/home/danielpedro/anaconda3/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [5]:
# Cria tabela fato unindo todas as dimensões
query = f"""
CREATE OR REPLACE TABLE `{table_id}` AS
SELECT
    -- f.order_id,
    ped.sk_order,
    dt.sk_date,
    f.order_date,  -- mantém da stg
    f.order_time,  -- mantém da stg
    pz.sk_pizza,
    
    f.quantity,
    f.unit_price,
    (f.unit_price * f.quantity) as total_price,

    -- PERIOD (corrigido para usar f.order_time diretamente)
    CASE 
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 12 AND 17 THEN 'Afternoon'
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 18 AND 22 THEN 'Evening'
        ELSE 'Late Night'
    END AS period,

    -- MEAL PERIOD
    CASE 
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 6 AND 9 THEN 'Breakfast'
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 10 AND 14 THEN 'Lunch'
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 15 AND 17 THEN 'Snacks'
        ELSE 'Dinner'
    END AS meal_period,

    -- SHIFT
    CASE 
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 6 AND 13 THEN '1st Shift'
        WHEN EXTRACT(HOUR FROM f.order_time) BETWEEN 14 AND 21 THEN '2nd Shift'
        ELSE '3rd Shift'
    END AS shift

FROM `{table_stg}` f
LEFT JOIN `{table_pizza}` pz
    ON f.pizza_name_id = pz.pizza_name_id
LEFT JOIN `{table_date}` dt
    ON f.order_date = dt.date  -- ⬅️ CORRIGIDO: provavelmente é 'date' ou 'full_date'

LEFT JOIN `{table_pedido}` ped
    ON f.order_id = ped.order_id
"""

In [6]:

try:
    # Executa a query que cria a tabela fato
    job = client.query(query)
    job.result()
    print("✅ Tabela fato criada com sucesso!")
    
    # Conta quantos registros tem na tabela
    check_query = f"SELECT COUNT(*) as total FROM `{table_id}`"
    result = client.query(check_query).result()
    for row in result:
        print(f"Total de registros na fato: {row.total}")
        
    # Pega uma amostra de 5 registros para verificação
    sample_query = f"""
    SELECT 
        sk_order,
        sk_date,
        sk_pizza,
        quantity,
        unit_price,
        total_price,
        period,
        meal_period,
        shift
    FROM `{table_id}` 
    LIMIT 5
    """
    sample_result = client.query(sample_query).result()
    
    # Mostra a amostra de dados
    print("\n📋 Amostra dos dados:")
    for i, row in enumerate(sample_result):
        print(f"Registro {i+1}: {dict(row)}")
        
except Exception as e:
    # Se der erro, mostra a mensagem
    print(f"❌ Erro ao criar tabela fato: {e}")

✅ Tabela fato criada com sucesso!
Total de registros na fato: 48620

📋 Amostra dos dados:
Registro 1: {'sk_order': 31, 'sk_date': 20150101, 'sk_pizza': 1, 'quantity': 1, 'unit_price': 9.75, 'total_price': 9.75, 'period': 'Afternoon', 'meal_period': 'Snacks', 'shift': '2nd Shift'}
Registro 2: {'sk_order': 80, 'sk_date': 20150102, 'sk_pizza': 1, 'quantity': 1, 'unit_price': 9.75, 'total_price': 9.75, 'period': 'Afternoon', 'meal_period': 'Lunch', 'shift': '1st Shift'}
Registro 3: {'sk_order': 91, 'sk_date': 20150102, 'sk_pizza': 1, 'quantity': 1, 'unit_price': 9.75, 'total_price': 9.75, 'period': 'Afternoon', 'meal_period': 'Snacks', 'shift': '2nd Shift'}
Registro 4: {'sk_order': 112, 'sk_date': 20150102, 'sk_pizza': 1, 'quantity': 1, 'unit_price': 9.75, 'total_price': 9.75, 'period': 'Evening', 'meal_period': 'Dinner', 'shift': '2nd Shift'}
Registro 5: {'sk_order': 132, 'sk_date': 20150102, 'sk_pizza': 1, 'quantity': 1, 'unit_price': 9.75, 'total_price': 9.75, 'period': 'Evening', 'meal